In [ ]:
import os
import pandas as pd
import sqlite3
from datetime import datetime

# Define local directory and SQLite database file
local_dir = "/content/drive/My Drive/nyc_taxi_data_2019"
db_file = "nyc_taxi_data_2019.db"

# Function to process data
def process_data(file_path, pickup_col, dropoff_col, distance_col, fare_col, passenger_col, taxi_type):
    # Read parquet file into DataFrame
    df = pd.read_parquet(file_path)

    # Select necessary columns and drop missing values
    df = df[[pickup_col, dropoff_col, distance_col, fare_col, passenger_col]].dropna()

    # Rename columns
    df.columns = ['pickup_datetime', 'dropoff_datetime', 'trip_distance', 'fare_amount', 'passenger_count']

    # Convert datetime columns
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

    # Derive new columns: trip duration (minutes) and average speed (mph)
    df['trip_duration'] = (df['dropoff_datetime'] - df['pickup_datetime']).dt.total_seconds() / 60
    df['average_speed'] = df['trip_distance'] / (df['trip_duration'] / 60)

    # Remove invalid data
    df = df[(df['trip_duration'] > 0) & (df['average_speed'].notnull())]

    # Add taxi type column
    df['taxi_type'] = taxi_type

    # Aggregate data: total trips, average fare, and total passengers per day
    df['pickup_date'] = df['pickup_datetime'].dt.date
    agg_df = df.groupby('pickup_date').agg(
        total_trips=('trip_distance', 'count'),
        average_fare=('fare_amount', 'mean'),
        total_passengers=('passenger_count', 'sum')
    ).reset_index()

    agg_df['taxi_type'] = taxi_type

    # Drop the pickup_date column from the main dataframe to avoid insertion errors
    df = df.drop(columns=['pickup_date'])

    return df, agg_df

# Function to load data into SQLite
def load_to_sqlite(df, table_name, conn):
    df.to_sql(table_name, conn, if_exists='append', index=False)

# Create a connection to the SQLite database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Drop tables if they exist (optional)
cursor.executescript('''
DROP TABLE IF EXISTS Trips;
DROP TABLE IF EXISTS DailyMetrics;
''')

# Create tables
cursor.executescript('''
-- Create Trips Table
CREATE TABLE IF NOT EXISTS Trips (
    trip_id INTEGER PRIMARY KEY AUTOINCREMENT,
    taxi_type TEXT,
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    trip_distance REAL,
    fare_amount REAL,
    passenger_count INTEGER,
    trip_duration REAL,
    average_speed REAL
);

-- Create DailyMetrics Table
CREATE TABLE IF NOT EXISTS DailyMetrics (
    metric_id INTEGER PRIMARY KEY AUTOINCREMENT,
    taxi_type TEXT,
    pickup_date DATE,
    total_trips INTEGER,
    average_fare REAL,
    total_passengers INTEGER
);
''')

# Process each file and load data into SQLite
for month in ['01']:
    for taxi_type, pickup_col, dropoff_col, distance_col, fare_col, passenger_col in [
        ('yellow', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'fare_amount', 'passenger_count'),
        ('green', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'trip_distance', 'fare_amount', 'passenger_count'),
        # ('fhvhv', 'pickup_datetime', 'dropoff_datetime', 'trip_miles', 'base_passenger_fare', 'passenger_count')
    ]:
        # Get file path
        file_path = os.path.join(local_dir, f'{taxi_type}_tripdata_2019-{month}.parquet')

        # Process data
        if os.path.exists(file_path):
            try:
                df, agg_df = process_data(file_path, pickup_col, dropoff_col, distance_col, fare_col, passenger_col, taxi_type)

                # Load processed data into SQLite
                load_to_sqlite(df, 'Trips', conn)

                # Load aggregated data into SQLite
                load_to_sqlite(agg_df, 'DailyMetrics', conn)

                print(f"Data for {taxi_type} 2019-{month} loaded successfully.")
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
        else:
            print(f"File {file_path} does not exist.")

# Commit and close the SQLite connection
conn.commit()
conn.close()

print("Data processing and loading to SQLite completed.")


Data for yellow 2019-01 loaded successfully.
Data for green 2019-01 loaded successfully.
Data processing and loading to SQLite completed.
